<a href="https://colab.research.google.com/github/JaeHoonKOR/NLP/blob/Colab/Co_occurrence_matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from nltk import FreqDist
import numpy as np
import re
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords 

sw = stopwords.words('english')

def buildDict(docs):
    doc_tokens = []     # python list
    for doc in docs:
        delim = re.compile(r'[\s,.]+')
        tokens = delim.split(doc.lower())
        tokens = [t for t in tokens if t not in sw] 
        if tokens[-1] == '' :   tokens = tokens[:-1] 
        doc_tokens.append(tokens)

    vocab = FreqDist(np.hstack(doc_tokens))
    vocab = vocab.most_common()
    word_to_id = {word[0] : id for id, word in enumerate(vocab)}
    id_to_word = {id : word[0] for id, word in enumerate(vocab)}
    corpus = np.array([id for id, _ in enumerate(vocab)])
    return doc_tokens, corpus, word_to_id, id_to_word

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [31]:
import pandas as pd
with open('/content//sample_data/bts_korean.txt', 'r') as f:
    documents = [line.strip() for line in f.readlines() if line.strip()]
for id, doc in enumerate(documents):
    print('[{}] : {}...'.format(id, doc[:30]))

doc_tokens, corpus, word_to_id, id_to_word = buildDict(docs)

[0] : 방탄복이 총알을 막아내는 것처럼, 살아가는 동안 힘든 ...
[1] : 2013년 방탄소년단은 《2 COOL 4 SKOOL》을...
[2] : 현재까지 방탄소년단은 전세계에서 3,000만 장 가량의...
[3] : 방탄소년단은 SNS를 통한 팬들과의 소통이 활발하여 2...
[4] : 빅히트 엔터테인먼트의 대표이자 프로듀서인 방시혁은 20...
[5] : 2013년...
[6] : 2013년의 방탄소년단 ....
[7] : 방탄소년단은 데뷔를 앞둔 2013년 5월 20일, 직접...
[8] : 2014년...
[9] : 2014년 3월 공연 중인 방탄소년단...
[10] : 2014년 1월 16일 경희대학교 평화의 전당에서 열린...
[11] : 2015년...
[12] : 2014년 11월 17일 인천 한류 콘서트에서의 방탄소...
[13] : 2015년 1월 15일 방탄소년단은 중국 북경 완스다 ...
[14] : 방탄소년단은 2015년 10월 15일 유럽 최대 음악 ...
[15] : 2016년...
[16] : 2016년 5월 7일 화양연화 프레스 컨퍼런스에 참석한...
[17] : 2016년 1월 14일 방탄소년단은 올림픽공원 체조경기...
[18] : 2016년 11월 16일 아시아 아티스트 어워즈에 참석...
[19] : 이후 6월부터 8월까지 대만과 마카오, 중국(난징, 베...
[20] : 2017년...
[21] : 2017년 1월 14일 골든디스크 어워드에 참석한 방탄...
[22] : BTS는 2017년 1월 13일 경기도 일산 킨텍스에서...
[23] : 2017년 11월 19일 아메리칸 뮤직 어워드에 참석한...
[24] : 2017년 9월 18일 방탄소년단은 다섯 번째 미니 앨...
[25] : 2017년 11월 13일 방탄소년단의 트위터 코리아 공...
[26] : 2018년...
[27] : 2018년 10월 24일 방탄소년단...
[28] : 방탄소년단은 2018년 1월 10일과 11일 경기도 일...
[29] : 2

In [32]:
def create_co_matrix(corpus, vocab_size, window_size=1):
    '''동시발생 행렬 생성
    :param corpus: 말뭉치(단어 ID 목록)
    :param vocab_size: 어휘 수
    :param window_size: 윈도우 크기(윈도우 크기가 1이면 타깃 단어 좌우 한 단어씩이 맥락에 포함)
    :return: 동시발생 행렬
    '''
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)

    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size + 1):
            left_idx = idx - i
            right_idx = idx + i

            if left_idx >= 0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] += 1

            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] += 1

    return co_matrix

In [33]:
def ppmi(C, verbose=False, eps = 1e-8):
    '''PPMI(점별 상호정보량) 생성
    :param C: 동시발생 행렬
    :param verbose: 진행 상황을 출력할지 여부
    :return:
    '''
    M = np.zeros_like(C, dtype=np.float32)
    N = np.sum(C)
    S = np.sum(C, axis=0)
    total = C.shape[0] * C.shape[1]
    cnt = 0

    for i in range(C.shape[0]):
        for j in range(C.shape[1]):
            pmi = np.log2(C[i, j] * N / (S[j]*S[i]) + eps)
            M[i, j] = max(0, pmi)

            if verbose:
                cnt += 1
                if cnt % (total//100 + 1) == 0:
                    print('%.1f%% 완료' % (100*cnt/total))
    return M

In [35]:
window_size = 2

vocab_size = len(word_to_id)
print('동시발생행렬 계산')
C = create_co_matrix(corpus, vocab_size, window_size)
W = ppmi(C)

print(C[0,:10])
print(W[0, :10])

동시발생행렬 계산
[0 1 1 0 0 0 0 0 0 0]
[ 0.       10.837365 10.422328  0.        0.        0.        0.
  0.        0.        0.      ]


In [37]:
from sklearn.utils.extmath import randomized_svd
wordvec_size = 100
U, S, V = randomized_svd(W, n_components=wordvec_size, n_iter=5, random_state=None)

In [38]:
from sklearn.metrics.pairwise import cosine_similarity
def most_similar(query, word_to_id, id_to_word, word_matrix, top=5):

    if query not in word_to_id:
        print('{}를 찾을 수 없음.'.format(query))
        return
        
    word_vector = np.array(word_matrix[word_to_id[query]]) #쿼리단어 벡터 추출
    word_vector = word_vector.reshape(1, -1) #cosine_similarity 위해 벡터 형상 조정
    sim = cosine_similarity(word_vector, word_matrix)
    sim = sim[0] #벡터 형상조정([[]] --> [])
    sim = [(id, cos) for id, cos in enumerate(sim)] #id, 유사도쌍으로 정리
    sim = sorted(sim, key=lambda x: x[1], reverse=True) #유사도 높은 순 정렬

    return sim[1:top+1]

In [40]:
rank = most_similar('월드', word_to_id, id_to_word, U)
for r in rank:
    print(id_to_word[r[0]], r[1])

무대에 0.99016315
15일 0.98953784
참석한 0.96215606
episode 0.9586951
공개했다 0.91906416
